 # Notebook to get abstracts from Semantic Scholar using DOIs, PMIDs, or PMCIDs 

In [14]:
import pandas as pd

In [15]:
inp_df = pd.read_csv('data/filtered_data_with_identifiers.csv')
print(inp_df.shape)
inp_df.head()

(8299, 23)


,Unnamed: 0,year,month,title,link_flair_text,domain,score,num_comments,sensationalism_score,jargon_proportion,...,is_top_domain_scientific,is_top_domain_news,is_top_domain_repo,is_top_domain_scam,is_top_domain_unknown,is_top_domain_indecisive,is_top_domain_less_than_2,label_voting_lm,label_voting_manual,identifier
0,5,2018,3,Firearm Injuries Drop 20 Percent When Gun Owne...,Biology,nejm.org,84,22,0.530595,0.000000,...,False,False,True,False,False,False,False,repo,repo,DOI:10.1056/NEJMc1712773
1,19,2018,3,Supplementation with probiotics during late pr...,Health,journals.plos.org,8,1,0.482136,0.314286,...,True,False,False,False,False,False,False,scientific,scientific,DOI:10.1371/journal.pmed.1002507
2,25,2018,3,Study finds that bee venom could be a useful p...,Medicine,ncbi.nlm.nih.gov,27,9,0.507827,0.333333,...,False,False,True,False,False,False,False,repo,repo,PMCID:PMC5793096
3,31,2018,3,"The interplay of gene flow, population size va...",Biology,onlinelibrary.wiley.com,6,0,0.433548,0.352941,...,True,False,False,False,False,False,False,scientific,scientific,DOI:10.1111/evo.13435/abstract
4,33,2018,3,Undisclosed Conflicts of Interests among Biome...,Social Science,ncbi.nlm.nih.gov,263,21,0.511708,0.086957,...,False,False,True,False,False,False,False,repo,repo,PMID:29400625


In [16]:
# Using semantic scholar bulk API to get abstracts
import requests
import time
import os

In [17]:
# TODO: Get the API key from the environment variable
SEM_SCHOLAR_API_KEY = os.getenv('SEM_SCHOLAR_API_KEY')

In [18]:
# API endpoint: https://api.semanticscholar.org/graph/v1/paper/batch
headers = {
    'x-api-key': SEM_SCHOLAR_API_KEY
}

def fetch_paper_details(ids):
    url = 'https://api.semanticscholar.org/graph/v1/paper/batch'
    params = {
        'fields': 'title,abstract'
    }
    response = requests.post(url, headers=headers, json={'ids': ids}, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} for IDs: {ids}")
        return []


batch_size = 100
results = []
# for i in range(0, len(inp_df), batch_size):
for i in range(0, 2, batch_size): # for testing purposes
    batch_ids = inp_df['identifier'].iloc[i:i + batch_size].tolist()
    data = fetch_paper_details(batch_ids)
    results.extend(data)
    time.sleep(1)  # Max 1 request per second


In [19]:
print(len(results))

100


In [20]:
results[0]

{'paperId': 'b7658773f34dc8247f9f768be58257d9014c7dd1',
 'title': 'Reduction in Firearm Injuries during NRA Annual Conventions.',
 'abstract': 'Decline in Firearm Injuries during NRA Conventions Analyses of a large national database show reductions in firearm injuries during annual conventions of the National Rifle Association, which are attended by thousands of firearm owners.'}

In [21]:
abstract_df = pd.DataFrame(results)

AttributeError: 'NoneType' object has no attribute 'keys'